# Exercise 3

### - Chosen datasets: ```Blood stem cell (GSE18067)```, ```Erythroid (GSE18067)```

####  Import libraries

In [18]:
import GEOparse
import pandas as pd

### Load the dataset

In [8]:
gse = GEOparse.get_GEO(geo="GSE18067", destdir="data/")

21-Aug-2024 20:15:23 DEBUG utils - Directory data/ already exists. Skipping.
21-Aug-2024 20:15:23 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE18nnn/GSE18067/soft/GSE18067_family.soft.gz to data/GSE18067_family.soft.gz
100%|██████████| 71.5M/71.5M [00:29<00:00, 2.55MB/s]  
21-Aug-2024 20:15:53 DEBUG downloader - Size validation passed
21-Aug-2024 20:15:53 DEBUG downloader - Moving /var/folders/6b/mjfxydrs1hb4_cf4_vhhd8lc0000gn/T/tmpfl1a4oq2 to /Users/shir/Documents/PycharmProjects/personal-medicine-tau/exercise3/data/GSE18067_family.soft.gz
21-Aug-2024 20:15:53 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE18nnn/GSE18067/soft/GSE18067_family.soft.gz
21-Aug-2024 20:15:53 INFO GEOparse - Parsing data/GSE18067_family.soft.gz: 
21-Aug-2024 20:15:53 DEBUG GEOparse - DATABASE: GeoMiame
21-Aug-2024 20:15:53 DEBUG GEOparse - SERIES: GSE18067
21-Aug-2024 20:15:53 DEBUG GEOparse - PLATFORM: GPL6238
21-Aug-2024 20:15:54 DEBUG GEOparse 

#### Inspect the dataset

In [66]:
print("GSM example:")
for gsm_name, gsm in gse.gsms.items():
    print("Name: ", gsm_name)
    print("Metadata:",)
    for key, value in gsm.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    print ("Table data:",)
    print (gsm.table.head())
    break

print()
print("GPL example:")
for gpl_name, gpl in gse.gpls.items():
    print("Name: ", gpl_name)
    print("Metadata:",)
    for key, value in gpl.metadata.items():
        print(" - %s : %s" % (key, ", ".join(value)))
    print("Table data:",)
    print(gpl.table.head())
    break

GSM example:
Name:  GSM451699
Metadata:
 - title : BXD13 Myeloid batch1
 - geo_accession : GSM451699
 - status : Public on Sep 12 2009
 - submission_date : Sep 10 2009
 - last_update_date : Sep 11 2009
 - type : RNA
 - channel_count : 1
 - source_name_ch1 : Myeloid
 - organism_ch1 : Mus musculus
 - taxid_ch1 : 10090
 - characteristics_ch1 : mouse id: 13, bxd strain: 13, cell type: Gr1+, array batch: 1
 - molecule_ch1 : total RNA
 - extract_protocol_ch1 : Total RNA was isolated using the RNeasy Mini kit (Qiagen) in accordance with the manufacturer’s protocol.
 - label_ch1 : biotin
 - label_protocol_ch1 : Biotinylated cRNA was prepared using the Illumina TotalPrep RNA Amplification Kit (Ambion) according to the manufacturer’s specifications starting with 100 ng total RNA
 - hyb_protocol : Standard Illumina hybridization protocol
 - scan_protocol : Standard Illumina scanning protocol
 - description : none
 - data_processing : Quantile normalization was applied on log2 transformed data.
 -

#### Extract expression data and annotations

In [116]:
# extract the samples and platform data
samples = gse.gsms
platform = gse.gpls['GPL6238']

# extract the expression data and convert into a DataFrame
expression_data = {}
for sample_name, sample in samples.items():
    title = sample.metadata['title'][0]
    strain = title.split(' ')[0]
    cell_type = title.split(' ')[1]
    technical_info_list = title.split(' ')[2:]
    technical_info = '_'.join(technical_info_list)
    
    # add the cell type to the expression data if it doesn't exist
    if cell_type not in expression_data.keys():
        expression_data[cell_type] = {}
    
    # add the strain to the cell type data
    expression_data[cell_type][f'{strain}_{technical_info}'] = sample.table.set_index('ID_REF')['VALUE']
        
# convert to DF
for cell_type, cell_data in expression_data.items():
    expression_data[cell_type] = pd.DataFrame(cell_data)

##### Merge with the platform annotations to get gene symbols or Entrez IDs

In [117]:
annotation_df = platform.table[['ID', 'Symbol']].copy()

# Merge the expression data with the annotation data
for cell_type, expression_df in expression_data.items():
    # name columns by strain
    expression_df.columns = [col.split('_')[0] for col in expression_df.columns]
    
    # merge
    merged_df = expression_df.merge(annotation_df, left_index=True, right_on='ID')
    
    # Set the gene symbol as the index and drop the ID column
    merged_df = merged_df.set_index('Symbol').drop('ID', axis=1)
    
    # Save the merged data
    expression_data[cell_type] = merged_df



### Data Preprocessing

#### Define the cell types of the analysis
The variable ```cell_types``` should be a list containing two of the following: ```['Progenitor', 'Stem', 'Erythroid', 'Myeloid']```

In [118]:
cell_types = ["Stem", "Erythroid"]

# get expression data for the desired cell types
preprocessed_data = {}
for cell_type in cell_types:
    preprocessed_data[cell_type] = expression_data[cell_type]

#### Preprocess

In [119]:
# set preprocessing params
maximal_val_threshold = 6.75
variance_threshold = 0.006

# load genotyping data
genotypes = pd.read_excel('data/genotypes.xls', skiprows=1, index_col=0, header=0)

# preprocess
for cell_type, expression_data in preprocessed_data.items():
    # remove rows lacking gene identifier
    expression_data = expression_data[expression_data.index.notnull()]
    
    # remove rows with low maximal value
    filtered_data = expression_data[expression_data.max(axis=1) > maximal_val_threshold]

    # remove rows with low variance
    filtered_data = filtered_data[filtered_data.var(axis=1) > variance_threshold]
    print(filtered_data)

    # for multiple probes of the same gene, keep the one with the highest variance
    filtered_data = filtered_data.groupby(filtered_data.index).apply(lambda x: x.loc[x.var(axis=1).idxmax()])

    # use only genes that are present in genotypes.xls
    #TODO: make sure that I didn't miss anything
    #common_strains = filtered_df.columns.intersection(genotype_data.columns)
    #filtered_df = filtered_df[common_strains]
    #genotype_data = genotype_data[common_strains]

    # average gene expression data across individuals of the same strain
    #averaged_df = filtered_df.groupby(filtered_df.columns, axis=1).mean()
    
    # save the processed dataset
    #averaged_df.to_csv(f'output/{cell_type}_processed_expression_data.csv')
    
    print(filtered_data)

                    BXD6     BXD28a      BXD19      BXD15      BXD40  \
Symbol                                                                 
Pigt           14.557293  14.324239  14.500165  14.834139  13.975895   
St6galnac3      6.825827   6.949788   7.037665   7.001356   6.922815   
Rangnrf        11.559403  10.742308  11.120819  10.725605  10.840271   
L1cam           6.677506   6.531395   6.829248   6.719764   6.579074   
Olfr159         6.786502   6.639528   6.755817   6.646977   6.677762   
...                  ...        ...        ...        ...        ...   
4930563H03Rik   6.656988   6.718578   6.604799   6.612627   6.711276   
2310051F07Rik  10.790007  10.489667  10.364562  10.104436   9.700218   
1110029L17Rik   9.037357   9.066524   8.995373   8.874153   9.089833   
1110059G02Rik   8.008045   7.925591   7.582184   7.551539   7.788641   
LOC333685       6.780722   6.890108   6.935633   6.878462   6.863248   

                   BXD12      BXD31       BXD9      BXD34     B

InvalidIndexError: Reindexing only valid with uniquely valued Index objects